In [6]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

import open3d as o3d

base_path = Path('./Dataset/')

metadata = pd.read_csv(base_path/'metadata_modelnet40.csv')

dataset_dir = base_path/'ModelNet40'


def read_off(file):
    if 'OFF' != file.readline().strip():
        raise ('Not a valid OFF header')
    n_verts, n_faces, __ = tuple(
        [int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')]
             for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:]
             for i_face in range(n_faces)]
    return verts, faces


# read in data
sample_label = 'airplane'
with open(f'{dataset_dir}/{sample_label}/train/{sample_label}_0001.off', 'r') as f:
    verts, faces = read_off(f)

In [7]:
class Texture:
    def __init__(self, img_path):
        self.img = cv2.imread(img_path)
        self.width = self.img.shape[1]
        self.height = self.img.shape[0]

    def get_color_value(self, u, v):
        x = int(u * self.width)
        y = int(v * self.height)
        return self.img[x, y]


texture = Texture('./cupe_uv.png')
texture.img.shape

(299, 400, 3)

In [8]:
v_max = np.max(verts, axis=0)
v_min = np.min(verts, axis=0)

In [9]:
# triangles_uv = []
# for face in faces:
#     # get the 3 vertices of the face
#     v1, v2, v3 = verts[face[0]], verts[face[1]], verts[face[2]]
#     # uv1 = [v1[0]/400, 1-v1[1]/200]
#     uv1 = [v1[0]/texture.width, 1-v1[1]/texture.height]
#     uv2 = [v2[0]/texture.width, 1-v2[1]/texture.height]
#     uv3 = [v3[0]/texture.width, 1-v3[1]/texture.height]
#     triangles_uv.append([uv1, uv2, uv3])

# triangles_uv = np.array(triangles_uv)
# print(triangles_uv.shape)  # (48, 2)

# u = x / texture_width
# v = (texture_height - y) / texture_height


In [10]:
def generate_texture_coords(verts, faces, texture_file):
    # Load the texture image
    texture = cv2.imread(texture_file)

    # Create an empty list to store the texture coordinates for each triangle
    triangles_uv = []

    # Loop over each face in the mesh's faces list
    for face in faces:
        # Retrieve the 3D vertex positions corresponding to the three vertices of the current face
        v1, v2, v3 = verts[face[0]], verts[face[1]], verts[face[2]]

        # Compute the texture coordinates for each vertex using the UV image
        uv1 = [v1[0] / texture.shape[1], (texture.shape[0] - v1[1]) / texture.shape[0]]
        uv2 = [v2[0] / texture.shape[1], (texture.shape[0] - v2[1]) / texture.shape[0]]
        uv3 = [v3[0] / texture.shape[1], (texture.shape[0] - v3[1]) / texture.shape[0]]

        # Append the texture coordinates for the three vertices of the current face to the list
        triangles_uv.extend([uv1, uv2, uv3])

    # Convert the list of texture coordinates to a numpy array of shape (num_faces*3, 2)
    triangles_uv = np.array(triangles_uv)

    return triangles_uv

texture_file = 'cupe_uv.png'

triangles_uv = generate_texture_coords(verts, faces, texture_file)
# triangles_uv = np.round(triangles_uv.clip(0,1) * 255).astype(np.uint8)
triangles_uv = triangles_uv.clip(0,1)
triangles_uv

array([[0.065583  , 0.86603612],
       [0.06280825, 0.86770234],
       [0.0637475 , 0.8696214 ],
       ...,
       [0.        , 0.84355385],
       [0.        , 0.84358261],
       [0.        , 0.84336421]])

In [11]:
m = o3d.geometry.TriangleMesh(o3d.open3d.utility.Vector3dVector(verts),
                              o3d.open3d.utility.Vector3iVector(faces))
m.compute_vertex_normals()
m.triangle_uvs = o3d.open3d.utility.Vector2dVector(triangles_uv)
m.triangle_material_ids = o3d.utility.IntVector([0]*len(faces))
m.textures = [o3d.geometry.Image(texture.img)]

o3d.visualization.draw_geometries([m])

: 

: 

[26.2332, 40.0552, 80.1577] [25.1233, 39.557, 77.4126] [25.499, 38.9832, 80.1355]


IndexError: index 10493 is out of bounds for axis 0 with size 299

In [23]:
import os
import cv2
import numpy as np
import open3d as o3d
 
vert = [[0,0,0],[0,1,0],[1,1,0],[1,0,0],
        [0,0,1],[0,1,1],[1,1,1],[1,0,1]]
        
faces = [[3, 0, 1], [1, 2, 3],
         [1, 5, 6], [6, 2, 1], 
         [5, 4, 7], [7, 6, 5], 
         [4, 0, 3], [3, 7, 4], 
         [6, 7, 3], [3, 2, 6], 
         [1, 0, 4], [4, 5, 1]]
 
m=o3d.geometry.TriangleMesh(o3d.open3d.utility.Vector3dVector(vert),
                            o3d.open3d.utility.Vector3iVector(faces))
 
m.compute_vertex_normals()
 
text=cv2.imread('./cupe_uv.png')
 
DX,DY=0.5/2,0.66/2 # UV坐标
 
one = [[3*DX,1*DY], [3*DX,2*DY], [4*DX,2*DY], [4*DX,2*DY], [4*DX,1*DY], [3*DX,1*DY]]
two = [[2*DX,1*DY], [2*DX,2*DY], [3*DX,2*DY], [3*DX,2*DY], [3*DX,1*DY], [2*DX,1*DY]]
three=[[1*DX,1*DY], [1*DX,2*DY], [2*DX,2*DY], [2*DX,2*DY], [2*DX,1*DY], [1*DX,1*DY]]
four= [[0*DX,1*DY], [0*DX,2*DY], [1*DX,2*DY], [1*DX,2*DY], [1*DX,1*DY], [0*DX,1*DY]]
five= [[1*DX,0*DY], [1*DX,1*DY], [2*DX,1*DY], [2*DX,1*DY], [2*DX,0*DY], [1*DX,0*DY]]
six = [[1*DX,2*DY], [1*DX,3*DY], [2*DX,3*DY], [2*DX,3*DY], [2*DX,2*DY], [1*DX,2*DY]]
 
v_uv=np.concatenate((one,two,three,four,five,six),axis=0)
# print(v_uv.shape) # (48, 2)
v_uv
# m.triangle_uvs = o3d.open3d.utility.Vector2dVector(v_uv)
# m.triangle_material_ids = o3d.utility.IntVector([0]*len(faces))
# m.textures = [o3d.geometry.Image(text)]
 
# o3d.visualization.draw_geometries([m])

array([[0.75, 0.33],
       [0.75, 0.66],
       [1.  , 0.66],
       [1.  , 0.66],
       [1.  , 0.33],
       [0.75, 0.33],
       [0.5 , 0.33],
       [0.5 , 0.66],
       [0.75, 0.66],
       [0.75, 0.66],
       [0.75, 0.33],
       [0.5 , 0.33],
       [0.25, 0.33],
       [0.25, 0.66],
       [0.5 , 0.66],
       [0.5 , 0.66],
       [0.5 , 0.33],
       [0.25, 0.33],
       [0.  , 0.33],
       [0.  , 0.66],
       [0.25, 0.66],
       [0.25, 0.66],
       [0.25, 0.33],
       [0.  , 0.33],
       [0.25, 0.  ],
       [0.25, 0.33],
       [0.5 , 0.33],
       [0.5 , 0.33],
       [0.5 , 0.  ],
       [0.25, 0.  ],
       [0.25, 0.66],
       [0.25, 0.99],
       [0.5 , 0.99],
       [0.5 , 0.99],
       [0.5 , 0.66],
       [0.25, 0.66]])

In [1]:
import torch
import tqdm
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("dream-textures/texture-diffusion", torch_dtype=torch.float16)
pipe = pipe.to("mps")

pipe.enable_attention_slicing()

sample_label = 'airplane'
prompt = f'smooth and aerodynamic {sample_label} surface'
image = pipe(prompt).images[0]  
image.save(f"Texture/{sample_label}.png")


/Users/keranwang/Project/3d-deep-learning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

: 